# Урок 3. Классификация. Логистическая регрессия.

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
X = np.array([[1, 1, 500, 1],
              [1, 1, 700, 1],
              [1, 2, 750, 2],
              [1, 5, 600, 1],
              [1, 3, 1450, 2],
              [1, 0, 800, 1],
              [1, 5, 1500, 3],
              [1, 10, 2000, 3],
              [1, 1, 450, 1],
              [1, 2, 1000, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [4]:
def standart_scale(X):
  mean = X.mean(axis = 0)
  std = X.std(axis = 0)
  return(X - mean) / std

In [5]:
X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standart_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [6]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    return - err / len(y)

In [7]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [8]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [9]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [10]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [11]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

In [12]:
W, error = eval_model(X_st, y, iterations=100, eta=1)

0 [ 0.46121499 -0.40908382  0.41426183  1.39855008] 0.760958797591889
10 [ 0.0387634  -1.30386446 -0.15001914  1.74758184] 0.4254600283700163
20 [-0.01161527 -1.64023671 -0.31196552  2.27533881] 0.38108858271134755
30 [-0.01156172 -1.88360488 -0.46035705  2.69723081] 0.3542235298084087
40 [ 9.33444488e-04 -2.07189803e+00 -5.97280973e-01  3.05064764e+00] 0.3358002141655348
50 [ 0.01934395 -2.22448855 -0.72422385  3.35653644] 0.3221745410588311
60 [ 0.04127542 -2.35245169 -0.84255667  3.6277315 ] 0.3115442516500474
70 [ 0.06539266 -2.46266676 -0.95347146  3.87262358] 0.3029143996576552
80 [ 0.090846   -2.55967079 -1.05796576  4.09697143] 0.2956909276718162
90 [ 0.11707191 -2.64659092 -1.15686395  4.30488038] 0.28949721467181433


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [13]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [14]:
calc_pred_proba(W, X_st)

array([0.32380055, 0.22296142, 0.96767742, 0.00787362, 0.65604029,
       0.36735518, 0.98703363, 0.14768606, 0.35244079, 0.94041789])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [15]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [16]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [17]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [18]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

True 	  [0 0 1 0 1 0 1 0 1 1]
Predicted [0 0 1 0 1 0 1 0 0 1]


In [19]:
accuracy(y, pred)

0.9

In [21]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: #TP
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: #TN
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: #FN
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: #FP
            cm[0][1] += 1
    return cm

In [22]:
cm = confusion_matrix(y, pred)
cm

array([[4., 0.],
       [1., 5.]])

In [23]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precision = TP / (TP + FP)
    return precision

In [24]:
precision(y, pred)

1.0

In [25]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    recall = TP / (TP + FN)
    return recall

In [26]:
recall(y, pred)

0.8

In [27]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [28]:
f_score(y, pred)

0.888888888888889

In [29]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


print(f'accuracy {accuracy_score(y, pred)}')
print(f'precision {precision_score(y, pred)}')
print(f'recall {recall_score(y, pred)}')
print(f'f-score {f1_score(y, pred)}')
print(f'\n {confusion_matrix(y, pred)}')

accuracy 0.9
precision 1.0
recall 0.8
f-score 0.888888888888889

 [[5 0]
 [1 4]]


6. Могла ли модель переобучиться? Почему?

Могла. В данном случае из-за маленькой выборки и большого кол-ва итераций. Из-за специфики логист. регрессии она склонна к переобучению.